In [ ]:
#@title
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from google.colab import drive



print('everything imported')
print('program is running')


dataDir = '/content/drive/MyDrive/Train2'
categories = ['Acne or Rosacea',
              'Acnitic Keratosis',
              'Eczema',
              'Melanoma',
              'Poison Ivy',
              'Psoriasis',
              'Seborrheic Keratoses',
              'Tinea Ringworm',
              'Warts, Molluscum, Other Viral Infections']

imgSize = 150

training_data = []
def vertical(filepath, classification, reps):
    img = load_img(filepath)
    data = img_to_array(img)
    samples = np.expand_dims(data, 0)
    datagen = ImageDataGenerator(height_shift_range=[-200, 200])
    it = datagen.flow(samples, batch_size=1)
    for i in range(reps):
        plt.subplot(330 + 1 + i)
        batch = it.next()
        image = batch[0].astype('uint8')
        data_image = img_to_array(image)
        data_resized = cv2.resize(data_image, (imgSize, imgSize))
        training_data.append([data_resized, classification])

def rotate(filepath, classification, reps):
    img = load_img(filepath)
    data = img_to_array(img)
    samples = np.expand_dims(data, 0)
    datagen = ImageDataGenerator(width_shift_range=[-200, 200])
    it = datagen.flow(samples, batch_size=1)
    for i in range(reps):
        plt.subplot(330 + 1 + i)
        batch = it.next()
        image = batch[0].astype('uint8')
        data_image = img_to_array(image)
        data_resized = cv2.resize(data_image, (imgSize, imgSize))
        training_data.append([data_resized, classification])

def create_training_data():
    for cat in categories:
        path = os.path.join(dataDir, cat)
        class_num = categories.index(cat)
        print(cat)
        count = 0
        for img in os.listdir(path):
            if count%100 == 0:
              print(count)
            img_array = cv2.imread(os.path.join(path, img))
            new_array = cv2.resize(img_array, (imgSize, imgSize))
            training_data.append([new_array, class_num])

            vertical(os.path.join(path, img), class_num, 4)

            rotate(os.path.join(path, img), class_num, 4)
            count += 1


create_training_data()
print('len of training data is:\n', len(training_data))


random.shuffle(training_data)


def convert_export(td):
    X = []
    y = []

    for features, label in td:
         X.append(features)
         y.append(label)

    X_aug = np.array(X).reshape(-1, imgSize, imgSize, 3)
    y_aug = np.array(y)
    return X_aug, y_aug

train, label = convert_export(training_data)

print('created training data')

Name = "5-128-7-CNN2"
tensorboard = TensorBoard(log_dir='logs\\test\\{}'.format(Name))

model = Sequential()
# 5 convolutional layers
model.add(Conv2D(64, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

# 5 Dense Layers
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))

# The output layer with 9 neurons, for 9 classes
model.add(Dense(9))
model.add(Activation("softmax"))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(train, label,
          batch_size=32,
          epochs=10,
          validation_split=0.1,
          callbacks=[tensorboard])


model.save('/content/drive/MyDrive/c.model')


everything imported
program is running
Acne or Rosacea
0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
Acnitic Keratosis
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
Eczema
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
